In [ ]:
# Import libraries
from torch import nn,cuda,backends, optim, save, load, no_grad
import torch
from torchvision.datasets import ImageFolder, CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import time
import pandas as pd

In [ ]:
# Image transformations
#mean=[0.6870, 0.5849, 0.5098]
#std=[0.2588, 0.3198, 0.3642]

mean=[0.4914, 0.4822, 0.4465]
std=[0.2023, 0.1994, 0.2010]

transformations=transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomInvert(p=0.2),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Normalize(mean=mean,std=std),
    ])

In [ ]:
batch_size=50
data_train = CIFAR10(root='./data', train=True, download=True, transform=transformations)
dataloader_train = torch.utils.data.DataLoader(data_train,batch_size=batch_size, shuffle=True, num_workers=2)

data_test = CIFAR10(root='./data', train=False, download=True, transform=transformations)
dataloader_test = torch.utils.data.DataLoader(data_test, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
# Get device for pytorch to run on
device = (
    "cuda"
    if cuda.is_available()
    else "mps"
    if backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

In [ ]:
class PrintLayer(nn.Module):
    def __init__(self,debug=""):
        super(PrintLayer, self).__init__()
        self.debug=debug
    
    def forward(self, x):
        # Do your print / debug stuff here
        print(self.debug,": ",x.shape)
        return x

In [ ]:
# CNN class
class NeuralNetwork(nn.Module):
    def __init__(self,output_features=141,rgb=True, dropout_percent=0.125,conv1_output_layers=36,conv2_output_layers=112):
        super().__init__()
        if (rgb):
            channels=3
        else:
            channels=1


        conv1=nn.Conv2d(channels,conv1_output_layers,3)
        relu=nn.ReLU()
        pool=nn.MaxPool2d(3,3)
        conv2=nn.Conv2d(conv1_output_layers,conv2_output_layers,9)
        #input_layer=nn.Linear(33*2*2, 25)
        #hl1=nn.Linear(25, 12)
        #hl2=nn.Linear(12, 9)
        #output_layer=nn.Linear(9, output_features)
        flatten=nn.Flatten()
        dropout=nn.Dropout(p=dropout_percent)
        softmax=nn.Softmax(dim=1)
        
        output_layer=nn.Linear(conv2_output_layers*2*2, output_features)
        #conv2=nn.Conv2d(36,output_features,9)

        

        self.filtering_relu_stack = nn.Sequential(
            conv1,
            relu,
            #dropout,
            pool,
            conv2,
            #dropout,
            #PrintLayer("After conv2"),
            relu,
            dropout,
            flatten,
        )

        self.linear_relu_stack = nn.Sequential(
            #input_layer,
            #relu,
            #dropout,
            #hl1,
            #relu,
            #dropout,
            #hl2,
            #relu,
            #dropout,
            output_layer,
            softmax,
        )

    def forward(self, x):
        x=self.filtering_relu_stack(x)
        
        #print(x.shape)
        #x=x.view(-1, 33*2*2)
        #print(x.shape)
        
        x=self.linear_relu_stack(x)

        return x


In [ ]:
def save_checkpoint(model,optimizer=None,filename="checkpoint.pt",params={}):
    params["model_state_dict"]=model.state_dict()
    if (optimizer is not None):
        params["optimizer_state_dict"]=optimizer.state_dict()
    save(params, filename)

In [ ]:
# Funtion to train model
def train_model(model, dataloader, checkpoint_data=None,criterion=None,optimizer=None, continue_from_epoch=0, epoches=5, limit_batches=-1,dataloader_test=None,path=""):
    if (criterion is None):
        criterion=nn.CrossEntropyLoss()
    if (optimizer is None):
        optimizer=optim.Adam(model.parameters(),lr=0.001)
    if (checkpoint_data is None):
        time_model=0
        best_epoch=(-1,999999999)
    else:
        time_model=checkpoint_data["time"]
        best_epoch=checkpoint_data["best_epoch"]
    f=open(path+"/model_stats.txt","w")

    for epoch in range(continue_from_epoch,continue_from_epoch+epoches):
        subloss=0
        total_loss=0
        start_time_epoch=time.time()
        start_time = time.time()
        for batch_number,(x_train,y_train) in enumerate(dataloader):
            x_train,y_train=x_train.to(device),y_train.to(device)
            y_pred=model(x_train)

            loss=criterion(y_pred,y_train)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #subloss+=loss.item()
            total_loss+=loss.item()
            #if (batch_number%100==0):
            #    print(f"Epoch: {epoch}, Batch Number: {batch_number}, Mini loss average: {subloss/100}, Mini training time: {time.time()-start_time} seconds")#, Model: {model}")
            #    subloss=0
            #    start_time = time.time()
            #if (limit_batches==batch_number):
            #    break
        stop_time=time.time()
        time_model=time_model+(stop_time-start_time_epoch)
        if (total_loss<best_epoch[1]):
            best_epoch=(epoch,total_loss)
        Accuracy_train=None
        Accuracy_test=None
        if (dataloader_test is not None):
            print ("Testing...")
            Accuracy_train=test_model(model,dataloader)
            Accuracy_test=test_model(model,dataloader_test)
        save_checkpoint(model,optimizer,filename=path+"/checkpoint2.pt",params={
            'epoch': epoch,
            'total_loss': total_loss,
            'time': time_model,
            'best_epoch': best_epoch,
            'Accuracy_test': Accuracy_test,
            'Accuracy_train': Accuracy_train,
            })
        print(f"Epoch {epoch} Loss: {total_loss}, Time: {stop_time-start_time_epoch} seconds, Accuracy on Test: {Accuracy_test}, Accuracy on Train: {Accuracy_train}")
        print(f"Epoch {epoch} Loss: {total_loss}, Time: {stop_time-start_time_epoch} seconds, Accuracy on Test: {Accuracy_test}, Accuracy on Train: {Accuracy_train}", file=f)
        if (abs(Accuracy_test-Accuracy_train)>0.12):
            print("Model has oveefitted. Stopping training...")
            print("Model has oveefitted. Stopping training...", file=f)
            break
    f.close()
    return best_epoch

# Function to test model
def test_model(model,dataloader,limit_batches=-1):
    samples_tested=0
    correct=0
    with no_grad():
        for batch_number,(x_test,y_test) in enumerate(dataloader):
            x_test,y_test=x_test.to(device),y_test.to(device)
            y_pred=model(x_test)


            samples_tested+=y_test.size(0)
            prediction=torch.max(y_pred,1)[1]
            correct+=(prediction==y_test).sum()
            #if (batch_number%100==0):
            #    print(f"Batch number: {batch_number}, Accuraccy: {correct/samples_tested}")
            #if (limit_batches==batch_number):
            #    break
    accuracy= correct/samples_tested
    return (accuracy)

def test_model_and_classes(model,dataloader,limit_batches=-1):
    samples_tested=0
    correct=0
    correct_per_class = {}
    total_per_class = {}
    with no_grad():
        for batch_number,(x_test,y_test) in enumerate(dataloader):
            x_test,y_test=x_test.to(device),y_test.to(device)
            y_pred=model(x_test)

            samples_tested+=y_test.size(0)
            prediction=torch.max(y_pred,1)[1]
            correct+=(prediction==y_test).sum()

            for label in range(y_test.size(0)):
                class_label = y_test[label].item()
                if class_label not in correct_per_class:
                    correct_per_class[class_label] = 0
                    total_per_class[class_label] = 0
                
                total_per_class[class_label] += 1
                if prediction[label] == y_test[label]:
                    correct_per_class[class_label] += 1
            #if (batch_number%100==0):
            #    print(f"Batch number: {batch_number}, Accuraccy: {correct/samples_tested}")
            #if (limit_batches==batch_number):
            #    break
    accuracy= correct/samples_tested
    for class_label in sorted(total_per_class.keys()):
        class_accuracy = correct_per_class[class_label] / total_per_class[class_label]
        print(f"Accuracy for class {class_label}: {class_accuracy:.4f}")

    return (accuracy)


In [ ]:
# Define model and optimizer

model=NeuralNetwork(output_features=len(data_train.classes),dropout_percent=0)
optimizer=optim.Adam(model.parameters(),lr=0.001)
def init(path="",CONTINUE_FROM_FILE=False,model=NeuralNetwork(output_features=len(data_train.classes),dropout_percent=0),optimizer=optim.Adam(model.parameters(),lr=0.001),continue_from=0,epoches=100):
    model.to(device)
    
    checkpoint_data={"time": 0, "best_epoch": (-1,99999999) }
# Continue from previous model, if CONTINUE_FROM_FILE is set to True
    if (CONTINUE_FROM_FILE):
        checkpoint = load(path+'/checkpoint.pt', map_location=torch.device(device))
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        checkpoint_data["time"]=checkpoint["time"]
        checkpoint_data["best_epoch"]=checkpoint["best_epoch"]
        continue_from=checkpoint["epoch"]+1
    print(checkpoint_data)
    train_model(model,dataloader_train, checkpoint_data=checkpoint_data, optimizer=optimizer, continue_from_epoch=continue_from, epoches=epoches, dataloader_test=dataloader_test,limit_batches=-1,path=path)
    with open(path+'/model.txt', 'w') as f:
        print(model, file=f)
        checkpoint = load(path+'/checkpoint2.pt', map_location=torch.device(device))
        print("\n\n",checkpoint, file=f)

        
#print (model)

In [ ]:
# Train Model
start = time.time()
best_epoch=train_model(model,dataloader_train, checkpoint_data=checkpoint_data, optimizer=optimizer, continue_from_epoch=continue_from, epoches=epoches, dataloader_test=dataloader_test,limit_batches=-1)
stop=time.time()
print(f"Total training time: {stop-start} seconds")

In [ ]:
# Test Model
accuracy_test=test_model(model,dataloader_test,limit_batches=-1)
accuracy_train=test_model(model,dataloader_train,limit_batches=-1)
print(f"Accuracy on Test: {accuracy_test}, Accuracy on Train: {accuracy_train}")

In [ ]:
# Calculate the mean and std of the dataset for normalization
mean = 0.
std = 0.
nb_samples = 0.
for data, _ in dataloader_train:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples
#file=open("/Downloads/mean_std.txt","w")
print("Mean: "+str(mean)+"\n"+"Std: "+str(std))
#file.write("Mean: "+str(mean)+"\n"+"Std: "+str(std))
#file.close()

In [ ]:
print(model)

In [ ]:
with open('model.txt', 'w') as f:
    print(model, file=f)

In [ ]:
max_epochs=100
dropout_percent=0.2
model=NeuralNetwork(output_features=len(data_train.classes),dropout_percent=dropout_percent,conv1_output_layers=100,conv2_output_layers=70)
optimizer=optim.Adam(model.parameters(),lr=0.001)
init(path="/home/i/ioakeime/results/adam/lr_0.001/conv1_output_layers_100_conv2_output_layers_70",CONTINUE_FROM_FILE=False,model=model,optimizer=optimizer,continue_from=0,epoches=max_epochs)

model=NeuralNetwork(output_features=len(data_train.classes),dropout_percent=dropout_percent,conv1_output_layers=36,conv2_output_layers=50)
optimizer=optim.Adam(model.parameters(),lr=0.001)
init(path="/home/i/ioakeime/results/adam/lr_0.001/conv1_output_layers_36_conv2_output_layers_50",CONTINUE_FROM_FILE=False,model=model,optimizer=optimizer,continue_from=0,epoches=max_epochs)

model=NeuralNetwork(output_features=len(data_train.classes),dropout_percent=dropout_percent,conv1_output_layers=100,conv2_output_layers=70)
optimizer=optim.Adam(model.parameters(),lr=0.0001)
init(path="/home/i/ioakeime/results/adam/lr_0.0001/conv1_output_layers_100_conv2_output_layers_70",CONTINUE_FROM_FILE=False,model=model,optimizer=optimizer,continue_from=0,epoches=max_epochs)

model=NeuralNetwork(output_features=len(data_train.classes),dropout_percent=dropout_percent,conv1_output_layers=36,conv2_output_layers=50)
optimizer=optim.Adam(model.parameters(),lr=0.0001)
init(path="/home/i/ioakeime/results/adam/lr_0.0001/conv1_output_layers_36_conv2_output_layers_50",CONTINUE_FROM_FILE=False,model=model,optimizer=optimizer,continue_from=0,epoches=max_epochs)


model=NeuralNetwork(output_features=len(data_train.classes),dropout_percent=dropout_percent,conv1_output_layers=100,conv2_output_layers=70)
optimizer=optim.SGD(model.parameters(), lr=0.001)
init(path="/home/i/ioakeime/results/sgd/lr_0.001/conv1_output_layers_100_conv2_output_layers_70",CONTINUE_FROM_FILE=False,model=model,optimizer=optimizer,continue_from=0,epoches=max_epochs)

model=NeuralNetwork(output_features=len(data_train.classes),dropout_percent=dropout_percent,conv1_output_layers=36,conv2_output_layers=50)
optimizer=optim.SGD(model.parameters(), lr=0.001)
init(path="/home/i/ioakeime/results/sgd/lr_0.001/conv1_output_layers_36_conv2_output_layers_50",CONTINUE_FROM_FILE=False,model=model,optimizer=optimizer,continue_from=0,epoches=max_epochs)

model=NeuralNetwork(output_features=len(data_train.classes),dropout_percent=dropout_percent,conv1_output_layers=100,conv2_output_layers=70)
optimizer=optim.SGD(model.parameters(), lr=0.0001)
init(path="/home/i/ioakeime/results/sgd/lr_0.0001/conv1_output_layers_100_conv2_output_layers_70",CONTINUE_FROM_FILE=False,model=model,optimizer=optimizer,continue_from=0,epoches=max_epochs)

model=NeuralNetwork(output_features=len(data_train.classes),dropout_percent=dropout_percent,conv1_output_layers=36,conv2_output_layers=50)
optimizer=optim.SGD(model.parameters(), lr=0.0001)
init(path="/home/i/ioakeime/results/sgd/lr_0.0001/conv1_output_layers_36_conv2_output_layers_50",CONTINUE_FROM_FILE=False,model=model,optimizer=optimizer,continue_from=0,epoches=max_epochs)


In [ ]:
model=NeuralNetwork(output_features=len(data_train.classes),dropout_percent=0.2,conv1_output_layers=100,conv2_output_layers=70)
model.to(device)
optimizer=optim.Adam(model.parameters(),lr=0.0001)
checkpoint = load('/home/i/ioakeime/results/adam/lr_0.0001/conv1_output_layers_100_conv2_output_layers_70/checkpoint2.pt', map_location=torch.device(device))
model.load_state_dict(checkpoint['model_state_dict'])

test_model_and_classes(model,dataloader_train,limit_batches=-1)
